In [88]:
import pandas as pd
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from os.path import join
from os import makedirs
from evaluator import Evaluator

In [89]:
MODEL_NAME = "EnsembleAugmentationCrossCovarianceMonitor30s"
MAX_SNR = np.inf
DETECTION_VECTOR_NAME = "fcov"
EXP_NAME="augmentation_ensemble_denoising"
EXP_RESULTS_DIR = f"../results/{EXP_NAME}"
EVALS_DIR = f"../evals/{EXP_NAME}/snr{MAX_SNR}/{method_fname(MODEL_NAME)}"

TRAIN_DATASETS = ["instance", "stead"]
TEST_DATASETS = ["instance", "stead"]
SPLITS = [0, 1, 2, 3, 4]
EPOCHS = range(-1, 20)

In [90]:
evals = []
for train_dataset in TRAIN_DATASETS:
    for test_dataset in TEST_DATASETS:
        
        for epoch in EPOCHS:
            evals.append({"train_dataset":train_dataset, "test_dataset":test_dataset, "model_name":MODEL_NAME, "epoch":epoch, "monitor_params":[DETECTION_VECTOR_NAME]})

In [91]:
df_metrics_list = []
for eval in evals:
    for split in SPLITS:
        f = h5.File(get_monitor_path(
                                    EXP_RESULTS_DIR,
                                    split_idx=split, 
                                      train_dataset=eval["train_dataset"], 
                                      test_dataset=eval["test_dataset"], 
                                      model_name=eval["model_name"],
                                      epoch=eval["epoch"],
                                      monitor_params=eval["monitor_params"]), "r")
            
        df_path = get_traces_path(
                        EXP_RESULTS_DIR,
                        split_idx=split, 
                        train_dataset = eval["train_dataset"], 
                        test_dataset=eval["test_dataset"], 
                        model_name = eval["model_name"])

        df_traces = pd.read_csv(df_path)

        detection_vector = np.array(f[DETECTION_VECTOR_NAME])
        

        # Reset traces index.
        df_traces.reset_index(inplace=True)
        df_eq = df_traces[df_traces.label == "eq"]
        df_no = df_traces[df_traces.label == "no"]

        # Eliminate ill cropped traces.
        if eval["test_dataset"] == "stead":
            _df_eq = df_eq.copy()

            _df_eq["snr_db"] = _df_eq["snr_db"].apply(lambda x: parse_stead_snr(x))
            # This is a list of lists.
            a = np.array(_df_eq["snr_db"].values.tolist())
            
            _df_eq["trace_E_snr_db"] = a[:, 0]  
            _df_eq["trace_N_snr_db"] = a[:, 1]
            _df_eq["trace_Z_snr_db"] = a[:, 2]

            df_eq = _df_eq.copy()

        df_eq = cropoffset_filter(df_eq)
        df_eq = snr_filter(df_eq, snr_max=MAX_SNR)

        # Concat eq and no traces. And shuffle them.
        df_traces = pd.concat([df_eq, df_no], ignore_index=False)
        df_traces = df_traces.sample(frac=1)

        f.close()

        assert np.any(np.isnan(detection_vector)) == False
        detection_vector = detection_vector[np.array(df_traces.index)]
        
        df_traces["detection_score"] = detector(detection_vector, DETECTION_VECTOR_NAME)
        fpr, tpr, thresholds1 = roc_curve(df_traces["label"] == "eq", df_traces["detection_score"])
        pre, rec, thresholds2 = precision_recall_curve(df_traces["label"] == "eq", df_traces["detection_score"])
        roc_auc = auc(fpr, tpr)
        print("roc_auc:", roc_auc)
        av_pre = average_precision_score(df_traces["label"] == "eq", df_traces["detection_score"])

        best_f1_threshold = get_maxf1_threshold(thresholds2, pre, rec)
        metrics = get_metrics(pre, rec, thresholds2, best_f1_threshold)
        metrics.update({"model":eval["model_name"], "epoch":eval["epoch"], "split":split, "training":eval["train_dataset"], "testing":eval["test_dataset"], "auc":roc_auc, "av_pre":av_pre})
        df_metrics_list.append(metrics)

df_metrics = pd.DataFrame(df_metrics_list)

training_sets = df_metrics["training"].unique()
test_sets = df_metrics["testing"].unique()

makedirs(EVALS_DIR, exist_ok=True)
df_metrics.to_csv(join(EVALS_DIR, "metrics.csv"), index=False)

roc_auc: 0.9712873648365208
roc_auc: 0.9714675275247662
roc_auc: 0.965876220916557
roc_auc: 0.9594890146520819
roc_auc: 0.9641455891223591
roc_auc: 0.962683038542915
roc_auc: 0.9621615898332538
roc_auc: 0.9595025024709947
roc_auc: 0.9601664279292914
roc_auc: 0.9393401525341599
roc_auc: 0.9609298197598103
roc_auc: 0.9674880241759161
roc_auc: 0.9652152659672294
roc_auc: 0.9687725474330506
roc_auc: 0.957590769637547
roc_auc: 0.953157502606728
roc_auc: 0.9592904786172831
roc_auc: 0.9634397357658213
roc_auc: 0.9700419072969246
roc_auc: 0.9503791478630519
roc_auc: 0.9490513074985035
roc_auc: 0.9450447181045492
roc_auc: 0.9407495411744892
roc_auc: 0.9726336256937612
roc_auc: 0.9350322617369664
roc_auc: 0.9516983526531138
roc_auc: 0.9390029884156385
roc_auc: 0.9522610030451334
roc_auc: 0.9690048473091559
roc_auc: 0.9429446869535965
roc_auc: 0.9599656539117165
roc_auc: 0.9496248005736827
roc_auc: 0.9519511994957649
roc_auc: 0.9722008379958837
roc_auc: 0.942919319216009
roc_auc: 0.97284524357024

/tmp/ipykernel_3109065/2228331604.py:20: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traces = pd.read_csv(df_path)


roc_auc: 0.989087074646878


/tmp/ipykernel_3109065/2228331604.py:20: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traces = pd.read_csv(df_path)


roc_auc: 0.9863485772401026


...

roc_auc: 0.987644765722841


/tmp/ipykernel_3109065/2228331604.py:20: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traces = pd.read_csv(df_path)


roc_auc: 0.9872652712706536


/tmp/ipykernel_3109065/2228331604.py:20: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traces = pd.read_csv(df_path)


roc_auc: 0.9873085509680805
